In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. 数据预处理和加载
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|█████████████████████████████████████████████████████████████████| 26421880/26421880 [00:15<00:00, 1725375.75it/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 135383.43it/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████| 4422102/4422102 [00:01<00:00, 2707972.88it/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 5099734.76it/s]

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



以上代码下载的数据集为FashinMNIST，有`10`个标签，依次为：
```python
0: T-shirt/top
1: Trouser
2: Pullover
3: Dress
4: Coat
5: Sandal
6: Shirt
7: Sneaker
8: Bag
9: Ankle boot
```


In [ ]:
下面的代码可能运行10分钟，使用的GPU如下信息如下。

In [ ]:
import torch

# 检测是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 输出当前设备是 CPU 还是 GPU
print(f'Using device: {device}')

# 如果使用 GPU，还可以查看 GPU 的信息
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))  # 输出 GPU 名称

In [20]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # 输入层（784），隐藏层1（128），隐藏层2（64），输出层（10 类）
        self.fc1 = nn.Linear(28 * 28, 128)  # W1, b1
        self.fc2 = nn.Linear(128, 64)       # W2, b2
        self.fc3 = nn.Linear(64, 10)        # W3, b3
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 将28x28的图像拉伸为1维
        x = torch.sigmoid(self.fc1(x))  # 激活函数 Sigmoid
        x = torch.relu(self.fc2(x))  # 激活函数 ReLU
        x = self.fc3(x)  # 输出层没有激活，因为我们使用了损失函数CrossEntropyLoss
        return x

model = SimpleNet()
criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失函数
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 使用随机梯度下降，学习率为0.01
num_epochs = 10  # 训练5个轮次

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        # 清除梯度
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

# 5. 测试网络
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

torch.save(model.state_dict(), 'simple_net.pth')

Epoch [1/10], Loss: 1.8241
Epoch [2/10], Loss: 0.9848
Epoch [3/10], Loss: 0.7343
Epoch [4/10], Loss: 0.6488
Epoch [5/10], Loss: 0.6031
Epoch [6/10], Loss: 0.5708
Epoch [7/10], Loss: 0.5447
Epoch [8/10], Loss: 0.5222
Epoch [9/10], Loss: 0.5034
Epoch [10/10], Loss: 0.4879
Accuracy: 81.53%


In [29]:
# 创建模型实例
model = SimpleNet()
# 加载模型参数
model.load_state_dict(torch.load(r"D:\machine_learning\DeepL\model\simple_net.pth"))
model.eval()  # 设置为评估模式

C:\Users\86151\AppData\Local\Temp\ipykernel_13396\299425768.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\machine_learning\DeepL\

SimpleNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [30]:
from PIL import Image
import torchvision.transforms as transforms

# 定义图片预处理
transform = transforms.Compose([
    transforms.Grayscale(),  # 转为灰度图
    transforms.Resize((28, 28)),  # 调整为28x28
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize((0.5,), (0.5,))  # 归一化
])

# 加载图片并预处理
image = Image.open(r"D:\machine_learning\DeepL\img\dress.jpg")  # 替换为你的图片路径
image = transform(image).unsqueeze(0)  # 添加批次维度

# 进行预测
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# 输出预测结果
print(f'Predicted class: {predicted.item()}')


Predicted class: 8


很显然，预测失败，训练的程度已经很好了，接下来就是超参数训练

*Created by Liu* `2024/10/15 19:30`